In [ ]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

In [ ]:
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

# Instantiate QDesign class and initialise user interface

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
design.chips.main.size['size_z'] = '-500um'
design.chips.main.size['sample_holder_bottom'] = '500um'
design.chips.main.size['sample_holder_top'] = '2000um'



# Set hfss as renderer for design
#hfss = design.renderers.hfss
# Allow overwriting
gui = MetalGUI(design)
design.overwrite_enabled = True

#design.delete_all_components()





In [ ]:
stg = ShortToGround(design, 'short_to_ground', options=dict(pos_x='-0.5mm',  pos_y='0um', orientation='180'))

In [ ]:
#Add hanger of capacitively coupled transmission lines
TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='-0.5mm',
                     pos_y='0mm',
                     coupling_length='200um',
                     coupling_space='1um',
                     rotation = '90'))

gui.rebuild()
gui.autoscale()

In [ ]:
otg1 = OpenToGround(design, 'open_to_ground', options=dict(pos_x='1mm',  pos_y='0um', orientation='0'))
rt_meander = RouteMeander(design, 'readout',  Dict(
        total_length='4.18 mm',
        hfss_wire_bonds = True,
        fillet='90 um',
        lead = dict(start_straight='100um'),
        pin_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='second_end'),
            end_pin=Dict(component='open_to_ground', pin='open')), ))

gui.rebuild()
gui.autoscale()

In [ ]:
#Setup the launchpad location and orientation.
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight


#launch_options1 = dict(pos_x='1.25mm', pos_y='0.9mm', orientation='180')

#lp = LaunchpadWirebond(design, 'P4_Q', options = launch_options1)

otg2 = OpenToGround(design, 'open_to_ground2', options=dict(pos_x='-0.5mm', pos_y='1.25mm',
orientation='0'))



#Setup the OpenToGround location and orientation.
#launch_options2 = dict(pos_x='-1.25mm',  pos_y='0.9mm', orientation='0')

#lp2 = LaunchpadWirebond(design, 'P5_Q', options = launch_options2)


otg3 = OpenToGround(design, 'open_to_ground3', options=dict(pos_x='-0.5mm', pos_y='-1.25mm',
orientation='180'))

#After the two QComponents are added to design, connect them with a RouteMeander.
meander_options_left = Dict(
        pin_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='prime_start'),
            end_pin=Dict(component=otg3.name, pin='open')), )

meander_options_right = Dict(
        pin_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='prime_end'),
            end_pin=Dict(component=otg2.name, pin='open')), )


meander_left = RouteStraight(design, 'cpw_left',options=meander_options_left)
meander_right = RouteStraight(design, 'cpw_right',options=meander_options_right)

gui.rebuild()
gui.autoscale()

In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_res = EPRanalysis(design, "hfss")

In [ ]:
hfss = eig_res.sim.renderer

In [ ]:
hfss.open_ansys(path = 'C:\Program Files\AnsysEM\AnsysEM19.3\Win64')

In [ ]:
hfss.connect_ansys()

In [ ]:
hfss.activate_ansys_design("Readout", 'eigenmode') 

In [ ]:
hfss.render_design(['short_to_ground', 'readout', 'open_to_ground', 'TQ1', otg2.name, otg3.name, 'cpw_left', 'cpw_right'], [])

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 20
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

# Next 2 lines are counterinuitive, since there is no junction in this resonator.
# However, these are necessary to make pyEPR work correctly. Please do note delete
hfss.pinfo.design.set_variable('Lj', '10 nH')
hfss.pinfo.design.set_variable('Cj', '0 fF')

setup.analyze()

In [ ]:
eig_res.sim.convergence_t, eig_res.sim.convergence_f, _ = hfss.get_convergences()
eig_res.sim.plot_convergences()

In [ ]:
hfss.disconnect_ansys()